In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import sklearn
import sklearn.discriminant_analysis
import sklearn.datasets

In [ ]:
n = 500
pct_test = 0.2
n_train = int(n*(1-pct_test))

X, y = sklearn.datasets.make_moons(500, shuffle=True, noise=0.3, random_state=0x2024_04_24)
X_train: np.ndarray = X[:n_train]
y_train: np.ndarray = y[:n_train]

X_test: np.ndarray = X[n_train:]
y_test: np.ndarray = y[n_train:]

colors = np.where(y == 0, "red", "blue")
plt.scatter(X[:, 0], X[:, 1], c=colors)

In [ ]:
qda = sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis(store_covariance=True)
qda.fit(X_train, y_train)

In [ ]:
np.mean(qda.predict(X_test) == y_test)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Step 2: Generate a grid of points
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 500),
                     np.linspace(y_min, y_max, 500))

# Step 3: Predict over the grid
Z = gnb.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Step 4: Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.5)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k')
plt.title("QDA Decision Boundary")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

In [ ]:
gnb.predict(X_test)

In [ ]:
import torch
import torch.nn as nn

In [ ]:
X_train.shape, y_train.shape

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

n = 1_000 
pct_test = 0.2
n_train = int(n*(1-pct_test))

X, y = sklearn.datasets.make_moons(n, shuffle=True, noise=0.35, random_state=0x2024_04_24)
X_train: np.ndarray = X[:n_train]
y_train: np.ndarray = y[:n_train]

X_test: np.ndarray = X[n_train:]
y_test: np.ndarray = y[n_train:]

colors = np.where(y == 0, "red", "blue")

# Assuming X_train and y_train are available as NumPy arrays
X_train_tensor: torch.Tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor: torch.Tensor = torch.tensor(y_train, dtype=torch.long)

# Create a dataset and data loader
dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset, batch_size=10, shuffle=True)  # You can adjust batch size

model = nn.Sequential(
    nn.LazyLinear(50),
    nn.ReLU(),
    nn.Linear(50, 2),
    nn.Softmax(dim = 1), 
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate as needed

num_epochs = 100  # Define the number of epochs

losses = []
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Reset gradients to zero for each batch
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()  # Backpropagate the error
        optimizer.step()  # Adjust weights

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

plt.plot(range(len(losses)), losses)

In [ ]:
model = nn.Sequential(
    nn.LazyLinear(50),
    nn.ReLU(),
    nn.Linear(50, 2),
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import sklearn.datasets

import itertools

for i, (n, num_epochs, num_nodes) in enumerate(itertools.product([1000], [100], [1, 2, 5, 25, 50, 100])):
    print(f"Running experiment {i+1} ({i / 27*100:.2f}% done)")
    print(f"{(n, num_epochs, num_nodes)=}")
    pct_test = 0.2
    n_train = int(n * (1 - pct_test))
    # X, y = sklearn.datasets.make_moons(n, shuffle=True, noise=0.35, random_state=0x20240424)
    X, y = sklearn.datasets.make_circles(n)
    X_train: np.ndarray = X[:n_train]
    y_train: np.ndarray = y[:n_train]
    X_test: np.ndarray = X[n_train:]
    y_test: np.ndarray = y[n_train:]

    # Converting to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)

    # DataLoader setup
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset, batch_size=10, shuffle=True)

    # Model definition
    model = nn.Sequential(
        nn.LazyLinear(num_nodes),
        nn.ReLU(),
        nn.LazyLinear(2)
    )

    # Loss function and optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1)

    # Training function
    def train(model, train_loader, loss_fn, optimizer, num_epochs=num_epochs):
        model.train()
        for epoch in range(num_epochs):
            for batch, (X_batch, y_batch) in enumerate(train_loader):
                pred = model(X_batch)
                loss = loss_fn(pred, y_batch)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    # Evaluation function
    def evaluate(model, X_test_tensor, y_test_tensor):
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            predicted_classes = predictions.argmax(1)
            accuracy = (predicted_classes == y_test_tensor).type(torch.float32).mean().item()
        return accuracy

    # Running the training and evaluation
    train(model, train_loader, loss_fn, optimizer, num_epochs=50)
    accuracy = evaluate(model, X_test_tensor, y_test_tensor)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    import numpy as np
    import matplotlib.pyplot as plt
    import torch

    # Dataset
    colors = np.where(y == 0, 'red', 'blue')

    # Create a mesh grid for the decision boundary
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    grid = np.c_[xx.ravel(), yy.ravel()]
    grid_tensor = torch.tensor(grid, dtype=torch.float32)

    # Predict over the grid
    model.eval()
    with torch.no_grad():
        preds = model(grid_tensor)
        preds = preds.argmax(1).numpy()

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(X[:, 0], X[:, 1], c=colors, edgecolors='k')
    contour = plt.contourf(xx, yy, preds.reshape(xx.shape), alpha=0.5, levels=np.arange(3)-0.5, colors=['red', 'blue'])
    plt.title(f"{n=}, {num_epochs=}, {num_nodes=}")
    plt.xlabel("Feature 1")
    plt.ylabel("Feature 2")
    plt.grid(True)
    plt.savefig(f"moon_{n}_{num_epochs}_{num_nodes}.png")

In [ ]:
X, y = make_moons(n, shuffle=True, noise=0.35, random_state=0x20240424)

In [ ]:
X, y = sklearn.datasets.make_circles(100)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)